In [14]:
import json
import conducto as co
from my_experiment import process_data, notebook_pkgs
data_dir = "/conducto/data/pipeline"

Make the root node of the tree.  It's a [Serial](/docs/basics/pipeline-structure#node-types) node, so Conducto runs its children one after another.

In [15]:
root = co.Serial()
root.describe()['id']

/

[Image](/docs/basics/Images) objects devine environments.

In [16]:
download_img = co.Image(install_packages=["wget"])

[Exec](/docs/basics/pipeline-structure#node-types) nodes run commands or call functions in those environments.

In [17]:
root["Download"] = co.Exec("wget -NcP {data_dir} http://192.168.90.13:8887/genedata.zip", 
                           image=download_img)
root.describe()['id']

/
└─0 Download   wget -NcP {data_dir} http://192.168.90.13:8887/genedata.zip

The Process and Analyze nodes share an image.  To build it:
 - start with a premade image from dockerhub
 - include the local directory so we can reference its other files
 - pip install their dependencies

In [18]:
bio_img = co.Image("ncbi/blast",
                   copy_dir=".",
                   install_pip=["pandas", "biopython"] + notebook_pkgs)

This node makes three calls to my_experiment.process_data(), each time with different parameters. 

In [19]:
process = co.Parallel(image=bio_img)
process["1"] = co.Exec(process_data, dataset=1, data_dir=data_dir)
process["2"] = co.Exec(process_data, dataset=2, data_dir=data_dir)
process["3"] = co.Exec(process_data, dataset=3, data_dir=data_dir)
root["Process"] = process

root.describe()['id']

/
├─0 Download   wget -NcP {data_dir} http://192.168.90.13:8887/genedata.zip
└─1 Process
  ├─ 1   conducto my_experiment.py process_data --dataset=1 --data_dir=/conducto/data/pipeline
  ├─ 2   conducto my_experiment.py process_data --dataset=2 --data_dir=/conducto/data/pipeline
  └─ 3   conducto my_experiment.py process_data --dataset=3 --data_dir=/conducto/data/pipeline

Once all our data is processed and ready to go, make a node to analyze it interactively.  Notebook nodes that run to completion can be viewed like reports.  Or you can leave them running and explore the data with code.

In [22]:
analyze = co.Notebook("analyze.ipynb", dir=data_dir, datasets=json.dumps([1,2,3]))

Add some extra resources for easy exploration.

In [23]:
analyze.set(image=bio_img, cpu=8, mem=32)
root["Analyze"] = analyze

root.describe()['id']

TreeError: Adding node Analyze violates the integrity of the pipeline

This will launch the pipeline and print a link.  The link will take you to the Conducto web app.  From there you can interact with the pipeline.

In [24]:
root.launch()

ValueError: Node references local code but the Image doesn't have enough information to infer the corresponding path inside the container.
Expected '.copy_dir', '.copy_url' inside a Git directory, or 'path_map'.  Node: /
  Image: {'name': 'gushy-ferroseed', 'image': 'ncbi/blast', 'dockerfile': None, 'dockerfile_text': None, 'docker_build_args': None, 'docker_auto_workdir': True, 'context': None, 'copy_repo': None, 'copy_dir': '.', 'copy_url': None, 'copy_branch': None, 'install_pip': ['pandas', 'biopython', 'conducto', 'jupyterlab', 'papermill', 'watchdog', 'ipdb', 'black'], 'install_npm': None, 'install_packages': None, 'install_docker': False, 'path_map': None, 'shell': '__auto__', 'instantiation_directory': '', 'reqs_py': ['pandas', 'biopython', 'conducto', 'jupyterlab', 'papermill', 'watchdog', 'ipdb', 'black'], 'reqs_npm': None, 'reqs_packages': None, 'reqs_docker': False}